In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import argparse
import json
import pandas as pd
import keras
import tensorflow as tf
assert(tf.__version__ == "1.15.2")
assert(keras.__version__ == "2.2.4")
import warnings
warnings.filterwarnings('ignore')

# Define working directory
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)
ROOT_DIR = os.path.abspath("../../")
sys.path.append(ROOT_DIR)

from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from utils.dataset import BloodCellDataset, BloodCellDatasetFromDataFrame
from utils.configs import BloodCellConfig_A, BloodCellConfig_B

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "maskrcnn-master","logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "maskrcnn-master", "mask_rcnn_coco.h5")

# Directory of images to run detection on
IMAGE_DIR_A = os.path.join(ROOT_DIR, "BCCD_Dataset/BCCD")

# Directory of images to run detection on
IMAGE_DIR_B = os.path.join(ROOT_DIR, "segmentation_WBC")

if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

IMAGE_COLLECTION_PATH = r'../blood_cell_dataset.pkl'

import imgaug as ia
import imgaug.augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)

Using TensorFlow backend.


In [3]:
if not os.path.exists(IMAGE_COLLECTION_PATH):
    raise Exception(f"IC_PATH @ {IMAGE_COLLECTION_PATH} not found.")
else:
    dataset = pd.read_pickle(IMAGE_COLLECTION_PATH)
    train_samples_A = dataset[(dataset["role"] == "train")&(dataset["origin"] == "A")]
    train_samples_B = dataset[(dataset["role"] == "train")&(dataset["origin"] == "B")]
    val_samples = dataset[dataset["role"] == "val"]

print(f"Number of samples in training set A: {len(train_samples_A)}")
print(f"Number of samples in training set B: {len(train_samples_B)}")
print(f"Number of samples in valid set: {len(val_samples)}")

Number of samples in training set A: 295
Number of samples in training set B: 100
Number of samples in valid set: 69


In [4]:
dataset_train_A = BloodCellDatasetFromDataFrame(train_samples_A, directory_path_A = IMAGE_DIR_A)
dataset_train_A.load_kernel()

dataset_train_B = BloodCellDatasetFromDataFrame(train_samples_B, directory_path_B = IMAGE_DIR_B)
dataset_train_B.load_kernel()

dataset_val = BloodCellDatasetFromDataFrame(val_samples, directory_path_A = IMAGE_DIR_A)
dataset_val.load_kernel()

In [5]:
config_A = BloodCellConfig_A()
config_A.STEPS_PER_EPOCH = 400
config_A.LEARNING_RATE = 5e-4
config_A.NAME = "BloodCell"

config_B = BloodCellConfig_B()
config_B.STEPS_PER_EPOCH = 100
config_B.LEARNING_RATE = 1e-4
config_B.NAME = "BloodCell"

In [6]:
aug_seq = iaa.Sequential(
    [
        # apply the following augmenters to most images
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.2), # vertically flip 20% of all images
        sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
            rotate=(-10, 10), # rotate by -45 to +45 degrees
            order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            cval=0, # if mode is constant, use a cval between 0 and 255
            mode=["constant"] # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        ))
    ])

# Stage A: train model on train_A (BBox and Classes)

In [7]:
reload = False
#model_branch = "bloodcell20201216T0822"
#checkpoint_epoch = 30

# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config_A,
                          model_dir=MODEL_DIR)
if not reload:
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])
else:
    model_path = os.path.join(MODEL_DIR, model_branch,
                                  f"mask_rcnn_bloodcell_{checkpoint_epoch:04d}.h5")
    # Load the last model you trained and continue training
    model.load_weights(model_path, by_name=True)







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead








In [8]:
checkpoint_path, checkpoint_epoch = model.train(dataset_train_A, dataset_val,
            learning_rate=config_A.LEARNING_RATE, augmentation=aug_seq,
            epochs=5,
            layers='heads')


Starting at epoch 0. LR=0.0005

Checkpoint Path: /home/jiayu.gan/Inarix/HEC/maskrcnn/maskrcnn-master/logs/bloodcell20201216T1102/mask_rcnn_bloodcell_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_con

In [9]:
checkpoint_path, checkpoint_epoch = model.train(dataset_train_A, dataset_val,
            learning_rate=config_A.LEARNING_RATE, augmentation=aug_seq,
            epochs=20,
            layers="all")


Starting at epoch 5. LR=0.0005

Checkpoint Path: /home/jiayu.gan/Inarix/HEC/maskrcnn/maskrcnn-master/logs/bloodcell20201216T1102/mask_rcnn_bloodcell_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a

Epoch 15/20
400/400 [==============================] - 212s 529ms/step - loss: 1.2043 - rpn_class_loss: 0.0200 - rpn_bbox_loss: 0.7895 - mrcnn_class_loss: 0.2263 - mrcnn_bbox_loss: 0.1684 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 1.1937 - val_rpn_class_loss: 0.0167 - val_rpn_bbox_loss: 0.7875 - val_mrcnn_class_loss: 0.1939 - val_mrcnn_bbox_loss: 0.1955 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 16/20
400/400 [==============================] - 210s 526ms/step - loss: 1.0960 - rpn_class_loss: 0.0191 - rpn_bbox_loss: 0.6975 - mrcnn_class_loss: 0.2249 - mrcnn_bbox_loss: 0.1545 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 1.2036 - val_rpn_class_loss: 0.0171 - val_rpn_bbox_loss: 0.7644 - val_mrcnn_class_loss: 0.2499 - val_mrcnn_bbox_loss: 0.1723 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 17/20
400/400 [==============================] - 211s 527ms/step - loss: 1.0715 - rpn_class_loss: 0.0190 - rpn_bbox_loss: 0.6944 - mrcnn_class_loss: 0.2070 - mrcnn_bbox_loss: 0.1510 - mrcnn_mask_loss: 0.0000e+00 - va

In [10]:
checkpoint_path, checkpoint_epoch = model.train(dataset_train_A, dataset_val,
            learning_rate=config_A.LEARNING_RATE/10, augmentation=aug_seq,
            epochs=60,
            layers="all")


Starting at epoch 20. LR=5e-05

Checkpoint Path: /home/jiayu.gan/Inarix/HEC/maskrcnn/maskrcnn-master/logs/bloodcell20201216T1102/mask_rcnn_bloodcell_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a

Epoch 30/60
400/400 [==============================] - 213s 532ms/step - loss: 0.9621 - rpn_class_loss: 0.0150 - rpn_bbox_loss: 0.6176 - mrcnn_class_loss: 0.1901 - mrcnn_bbox_loss: 0.1395 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 0.9139 - val_rpn_class_loss: 0.0135 - val_rpn_bbox_loss: 0.5842 - val_mrcnn_class_loss: 0.1608 - val_mrcnn_bbox_loss: 0.1553 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 31/60
400/400 [==============================] - 211s 528ms/step - loss: 0.9612 - rpn_class_loss: 0.0174 - rpn_bbox_loss: 0.6100 - mrcnn_class_loss: 0.1984 - mrcnn_bbox_loss: 0.1354 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 0.8200 - val_rpn_class_loss: 0.0107 - val_rpn_bbox_loss: 0.4732 - val_mrcnn_class_loss: 0.1856 - val_mrcnn_bbox_loss: 0.1504 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 32/60
400/400 [==============================] - 212s 529ms/step - loss: 0.9277 - rpn_class_loss: 0.0154 - rpn_bbox_loss: 0.6032 - mrcnn_class_loss: 0.1806 - mrcnn_bbox_loss: 0.1284 - mrcnn_mask_loss: 0.0000e+00 - va

400/400 [==============================] - 213s 533ms/step - loss: 0.9041 - rpn_class_loss: 0.0155 - rpn_bbox_loss: 0.5879 - mrcnn_class_loss: 0.1761 - mrcnn_bbox_loss: 0.1245 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 0.6623 - val_rpn_class_loss: 0.0077 - val_rpn_bbox_loss: 0.3855 - val_mrcnn_class_loss: 0.1530 - val_mrcnn_bbox_loss: 0.1162 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 52/60
400/400 [==============================] - 213s 532ms/step - loss: 0.8548 - rpn_class_loss: 0.0154 - rpn_bbox_loss: 0.5388 - mrcnn_class_loss: 0.1749 - mrcnn_bbox_loss: 0.1257 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 0.9222 - val_rpn_class_loss: 0.0174 - val_rpn_bbox_loss: 0.6411 - val_mrcnn_class_loss: 0.1493 - val_mrcnn_bbox_loss: 0.1143 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 53/60
400/400 [==============================] - 213s 532ms/step - loss: 0.8724 - rpn_class_loss: 0.0151 - rpn_bbox_loss: 0.5497 - mrcnn_class_loss: 0.1758 - mrcnn_bbox_loss: 0.1318 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 1.12

# Stage B: train model on train_B (mask only)

In [11]:
model_branch = checkpoint_path.split('/')[-2]
checkpoint_epoch = checkpoint_epoch

#model_branch = "bloodcell20201216T0822"
#checkpoint_epoch = 30

del model
model = modellib.MaskRCNN(mode="training", config=config_B,
                          model_dir=MODEL_DIR)
model_path = os.path.join(MODEL_DIR, model_branch,
                              f"mask_rcnn_bloodcell_{checkpoint_epoch:04d}.h5")
# Load the last model you trained and continue training
model.load_weights(model_path, by_name=True)
checkpoint_path, checkpoint_epoch = model.train(dataset_train_B, dataset_val,
                                                learning_rate=config_B.LEARNING_RATE, augmentation=aug_seq,
                                                epochs=100,
                                                layers="mrcnn_mask_heads")

Re-starting from epoch 60

Starting at epoch 60. LR=0.0001

Checkpoint Path: /home/jiayu.gan/Inarix/HEC/maskrcnn/maskrcnn-master/logs/bloodcell20201216T1102/mask_rcnn_bloodcell_{epoch:04d}.h5
Selecting layers to train
In model:  rpn_model
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_mask_deconv      (TimeDistributed)
Multiprocessing? True, number of workers 8
Epoch 61/100
100/100 [==============================] - 35s 352ms/step - loss: 0.6512 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.0000e+00 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.6512 - val_loss: 0.2991 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.0000e+00 

100/100 [==============================] - 27s 265ms/step - loss: 0.1050 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.0000e+00 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.1050 - val_loss: 0.4026 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.0000e+00 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.4026
Epoch 80/100
100/100 [==============================] - 27s 273ms/step - loss: 0.0854 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.0000e+00 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0854 - val_loss: 0.4352 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.0000e+00 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.4352
Epoch 81/100
100/100 [==============================] - 27s 271ms/step - loss: 0.0885 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.0000e+00 - mrcnn_bbox_

Epoch 99/100
100/100 [==============================] - 27s 268ms/step - loss: 0.0853 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.0000e+00 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0853 - val_loss: 0.4657 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.0000e+00 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.4657
Epoch 100/100
100/100 [==============================] - 27s 272ms/step - loss: 0.0792 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.0000e+00 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0792 - val_loss: 0.5668 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.0000e+00 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.5668
